In [1]:
import pandas as pd
from realtabformer import REaLTabFormer
from pytorch_tabnet.tab_model import TabNetClassifier
import torch
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from sklearn.metrics import balanced_accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import SMOTENC
from sklearn.utils import shuffle
from sklearn.tree import export_graphviz
from sklearn import metrics
from sklearn import tree
from xgboost import plot_tree
import pandas as pd
import numpy as np
import os
import wget
from pathlib import Path
import shutil
import gzip
import joblib
import pydot

random_state=42
np.random.seed(random_state)

from matplotlib import pyplot as plt

In [2]:
df = pd.read_csv('../../data/DNN-EdgeIIoT-dataset_SMALL.csv', low_memory=False)

In [3]:
drop_columns = ["frame.time", "ip.src_host", "ip.dst_host", "arp.src.proto_ipv4","arp.dst.proto_ipv4", 
         "http.file_data","http.request.full_uri","icmp.transmit_timestamp",
         "http.request.uri.query", "tcp.options","tcp.payload","tcp.srcport",
         "tcp.dstport", "udp.port", "mqtt.msg"]

df.drop(drop_columns, axis=1, inplace=True)
df.dropna(axis=0, how='any', inplace=True)
df.drop_duplicates(subset=None, keep="first", inplace=True)
df = shuffle(df)

categorical_columns = []
for col in df.columns[df.dtypes == object]:
    if col != "Attack_type":
        categorical_columns.append(col)

featuresFromStart = [ col for col in df.columns if col not in ["Attack_label"]+["Attack_type"]]

In [4]:
#rtf_model = REaLTabFormer(
#    model_type="tabular",
#    gradient_accumulation_steps=4,
#    logging_steps=100)

rtf_model = REaLTabFormer(
    model_type="tabular",
    batch_size=64,
    epochs=100,
    gradient_accumulation_steps=4,
    output_max_length=None,
    numeric_max_len=11,
    logging_steps=10)

In [5]:
rtf_model.fit(df, num_bootstrap=10)

Computing the sensitivity threshold...
Using parallel computation!!!


/Users/melicias/.local/lib/python3.7/site-packages/realtabformer/realtabformer.py:76: UserWarning: The device=cuda is not available, using device=mps instead.
  warnings.warn(f"The device={device} is not available, using device={_device} instead.")
/Users/melicias/.local/lib/python3.7/site-packages/realtabformer/realtabformer.py:568: UserWarning: Duplicate rate (0.0) in the data is zero. The `qt_interval` will be set                     to qt_interval_unique=100.
  to qt_interval_unique={qt_interval_unique}."


Bootstrap round:   0%|          | 0/10 [00:00<?, ?it/s]

Sensitivity threshold summary:
count    10.000000
mean     -0.000303
std       0.003365
min      -0.005734
25%      -0.003040
50%       0.000593
75%       0.001718
max       0.004237
dtype: float64
Sensitivity threshold: 0.004077137870855148 qt_max: 0.05


Map:   0%|          | 0/13893 [00:00<?, ? examples/s]

/Users/melicias/.local/lib/python3.7/site-packages/transformers/optimization.py:395: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


  0%|          | 0/270 [00:00<?, ?it/s]

{'loss': 3.3267, 'learning_rate': 4.9907407407407406e-05, 'epoch': 0.18}
{'loss': 0.9294, 'learning_rate': 4.981481481481482e-05, 'epoch': 0.37}
{'loss': 0.6637, 'learning_rate': 4.972222222222223e-05, 'epoch': 0.55}
{'loss': 0.5674, 'learning_rate': 4.962962962962963e-05, 'epoch': 0.73}
{'loss': 0.5136, 'learning_rate': 4.9537037037037035e-05, 'epoch': 0.92}
{'loss': 0.483, 'learning_rate': 4.9444444444444446e-05, 'epoch': 1.1}
{'loss': 0.457, 'learning_rate': 4.935185185185186e-05, 'epoch': 1.28}
{'loss': 0.436, 'learning_rate': 4.925925925925926e-05, 'epoch': 1.47}
{'loss': 0.4176, 'learning_rate': 4.9166666666666665e-05, 'epoch': 1.65}
{'loss': 0.4041, 'learning_rate': 4.9074074074074075e-05, 'epoch': 1.83}


KeyboardInterrupt: 

In [ ]:
rtf_model.save("rtf_model/")